In [1]:
import numpy as np
import pandas as pd

In [2]:
# import of dataset

train=pd.read_csv('./titanic dataset/train.csv')
test=pd.read_csv('./titanic dataset/test.csv')


In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
colums_to_drop=['Name','Cabin','Ticket','Embarked','PassengerId']
data=train.drop(colums_to_drop,axis=1)

In [39]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

data['Sex']=le.fit_transform(data['Sex'])

In [151]:
features=['Pclass','Sex','Age','SibSp','Parch','Fare']
class DecisionTree:
    def __init__(self,depth=0,maxDepth=5):
        self.left=None
        self.right=None
        self.attribute=None
        self.threshold=None
        self.depth=depth
        self.maxDepth=maxDepth
        self.label=None
    def entropy(self,col):
        tot=np.unique(col,return_counts=True)
        N=float(col.shape[0])
        
        ent=0.0
        
        for i in tot[1]:
            p=i/N
            ent+=(-1.0*(p*np.log2(p)))
        
        return ent
    def divide_data(self,x,attr,threshold):
        
        x_left=pd.DataFrame([],columns=x.columns)
        x_right=pd.DataFrame([],columns=x.columns)
        
        for i in range(x.shape[0]):
            val=x[attr].loc[i]
            if val>threshold:
#                 print("here")
                x_right=x_right.append(x.loc[i])
            else:
                x_left=x_left.append(x.loc[i])
        
        return x_left,x_right
            
    def info_gain(self,x,f,threshold):
        
#         print(f,threshold)
        left,right=self.divide_data(x,f,threshold)
        
        l=float(left.shape[0])/(x.shape[0])
        r=float(right.shape[0])/(x.shape[0])
        
#         dont want this
        if l==0 or r==0:
            return -100000
        InfoI=self.entropy(x.Survived);        
        InfoF=l*(self.entropy(left.Survived))+r*(self.entropy(right.Survived))
        
        return InfoI-InfoF
        
        
    
    def fit(self,x):
        
        n=x.shape[1]
        info_gains=[]
        for f in features:
            ig=self.info_gain(x,f,x[f].mean())
            
            info_gains.append(ig)
        print(info_gains)
        attr=np.argmax(info_gains)
        self.attribute=attr
#         for now we are considering the the mean as the threshold
        print('Creating Node...',features[self.attribute])
        self.threshold=x[features[attr]].mean()
        
        x_left,x_right=self.divide_data(x,features[self.attribute],self.threshold)
        x_left=x_left.reset_index(drop=True)
        x_right=x_right.reset_index(drop=True)
        
#         just setting the label of every node
        if x.Survived.mean()>=0.5:
            self.label=1
        else:
            self.label=0
        
#         pre pruning
        if x_left.shape[0]==0 or x_right.shape[0]==0:
            return
        
        if self.depth>=self.maxDepth:
            return
        
        
        self.left=DecisionTree(self.depth+1)
        
        self.right=DecisionTree(self.depth+1)

        self.left.fit(x_left)
        
        self.right.fit(x_right)
        
        return
        
        
    def predict(self,test):
        
        attr=self.attribute
        print(attr)
        if x[features[attr]]>self.threshold:
            
            if self.right is None:
                return self.label
            self.right.predict(test)
        else:
            if self.left is None:
                return self.label
            self.left.predict(test)
        
        
            
            
        

In [152]:
DT=DecisionTree()

In [154]:
DT.fit(data)

[0.07579362743608165, 0.2176601066606142, 0.0008836151229467681, 0.009584541813400071, 0.015380754493137694, 0.042140692838995464]
Creating Node... Sex
[0.20330629445826054, -100000, 0.007949583903816504, 0.009577910022389502, 0.013022528230396846, 0.059170413524787246]
Creating Node... Pclass
[0.007879174266377464, -100000, 1.1761177535707379e-05, 0.0017041154218443433, 0.00011507226891294131, 0.005040025507048629]
Creating Node... Pclass
[-100000, -100000, 0.001775915696611452, 0.0034102668912577294, 0.012074547098991278, 0.009105530540202772]
Creating Node... Parch
[-100000, -100000, 0.02338961625335166, 0.014416158684729466, -100000, 0.007514127745594262]
Creating Node... Age
[-100000, -100000, 0.0, 0.0, -100000, 0.0]
Creating Node... Age
[-100000, -100000, 0.0478244575284395, 0.042114225028137026, -100000, 0.01576728932977839]
Creating Node... Age
[-100000, -100000, 0.06345829864830327, 0.0701045817449687, 0.08496038399395706, 0.08496038399395706]
Creating Node... Parch
[-100000, 

In [148]:
test_data=test.drop(colums_to_drop,axis=1)

In [149]:
test_data=data.fillna(int(test_data['Age'].mean()))

In [153]:
y_=[]
for ix in range(test_data.shape[0]):
    y_.append(DT.predict(test_data.loc[ix]))

None


TypeError: list indices must be integers or slices, not NoneType